## Задания

### Задание 1
Тинькофф.Таргет - это агрегатор кэшбэков от партнеров (супермаркеты, магазины электроники, онлайн-магазины, заправки и т.п.).  

Под каждого партнера собирается целевая аудитория и запускается таргет на ограниченный период, поэтому у каждого клиента свой набор офферов и они периодически меняются.  

Механика: когда клиент совершает покупку по нашей карте в магазине X и у него есть оффер от этого партнера, он получает часть денег (какой-то фиксированный процент) за эту операцию обратно на карту.  

Одна из основных целей этого сервиса - чтобы как можно больше операций клиентов проходили через офферы.  

Например, если клиент хочет купить велосипед, то он сначала идет в наш сервис, смотрит есть ли у него кэшбэк в таком магазине и покупает велосипед по офферу.  

Плохой сценарий: если у клиента нет подходящего оффера или клиент его не находит среди других (у каждого клиента в моменте 200-400 офферов).  

**Придумайте 2 кейса-гипотезы для достижения основной цели и опишите процесс проведения исследования и АВ теста для проверки этой гипотезы.**  

#### Решение

**Кейс 1.**  
Предложить пользователям пройти опрос какие категории кэшбэка их интересуют больше всего. Разделить пользователей на две группы: А - рассказавшие о своих предпочтениях; Б - не ставшие отвечать на вопросы опроса. После группе А предлагать офферы максимально релевантные их интересам. Группе Б предлагать случайные офферы.  

**Нулевая гипотеза.** Количество операций клиентом через офферы не изменилось после участия в опросе.  

**Альтернативная гипотеза.** Участие в опросе и получение офферов совпадающих с интересами пользователя статистически значимо изменяет количество операций клиентом по офферам.

**Кейс 2.** 

Проанализировать транзакции пользователей за предыдущий период. Разделить пользователей на две группы: А - пользователи с проанализированными транзакциями; Б - пользователи анализ транзакций которых не проводился. После группе А предлагать офферы коррелирующие с их предыдущей историей транзакций. Группе Б предлагать случайные офферы.

**Нулевая гипотеза.** Количество операций по офферам клиентов из группы А и клиентов из группы Б равны.

**Альтернативная гипотеза.** Анализ транзакций пользователей за предыдущий период статистически значимо изменяет количество операций клиентом по офферам.

### Задание 2  
Маркетологи тестировали тему email-письма (разным группам послали email с разными темами).  

|     | Послали email  | Открыли email |
| ----- | ------------- | ------------- |
| Группа A | 10000 | 2200  |
| Группа B | 10000  | 2000  |

Целевой метрикой был процент открытий.

**Какой теме стоит отдать предпочтение и почему? Объясните свое решение**

#### Решение

In [208]:
# импорт библиотек
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

In [209]:
# задаем параметры групп
gr1_opened = 2200
gr2_opened = 2000
total = 10000

In [210]:
# моделируем группу А
gr1_ones = np.ones(gr1_opened, dtype='int')
gr1_zeros = np.zeros(total - gr1_opened, dtype='int')
group1_label = ['group A' for x in range(10000)]
group1 = np.concatenate((gr1_ones, gr1_zeros))
np.random.shuffle(group1)

In [211]:
# моделируем группу B
gr2_ones = np.ones(gr2_opened, dtype='int')
gr2_zeros = np.zeros(total-gr2_opened, dtype='int')
group2_label = ['group B' for x in range(10000)]
group2 = np.concatenate((gr2_ones, gr2_zeros))
np.random.shuffle(group2)

In [212]:
# создаем датафрейм
df1 = pd.DataFrame({'value': group1, 'group': group1_label})
df2 = pd.DataFrame({'value': group2, 'group': group2_label})

df = pd.concat([df1, df2], axis=0).reset_index(drop=True)

In [213]:
# перекрестная таблица
pd.crosstab(index=df.group, columns=df.value)

value,0,1
group,,
group A,7800,2200
group B,8000,2000


In [214]:
# воспользуемся chi square, т.к. имеем дело с дискретными данными
stat, p, dof, expected = chi2_contingency(pd.crosstab(index=df.group, columns=df.value))

In [215]:
# проводим t-test
alpha = 0.05  # уровень значимости

# вывод результата
if p <= alpha:
    print('Отклоняем нулевую гипотезу о равенстве двух групп. И принимаем альтернативную гипотезу о наличии статистически значимых различий.')
    if df[df['group'] == 'group A'].value.mean() > df[df['group'] == 'group B'].value.mean():
        print('Отдаем предпочтение группе A.')
    else:
        print('Отдаем предпочтение группе B.')
else:
    print('Не удалось выявить статистически значимые различия и отклонить нулевую гипотезу.\nНет разницы какой группе отдать предпочтение, т.к. они равны.')

Отклоняем нулевую гипотезу о равенстве двух групп. И принимаем альтернативную гипотезу о наличии статистически значимых различий.
Отдаем предпочтение группе A.


### Задание 3 
Маркетологи тестировали тему email-письма (разным группам послали email с разными темами).  

|     | Послали email  | Открыли email |
| ----- | ------------- | ------------- |
| Группа A | 10000 | 2070  |
| Группа B | 10000  | 2000  |

Целевой метрикой был процент открытий.

**Какой теме стоит отдать предпочтение и почему? Объясните свое решение**

#### Решение

In [216]:
# импорт библиотек
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

In [217]:
# задаем параметры групп
gr1_opened = 2070
gr2_opened = 2000
total = 10000

In [218]:
# моделируем группу А
gr1_ones = np.ones(gr1_opened, dtype='int')
gr1_zeros = np.zeros(total - gr1_opened, dtype='int')
group1_label = ['group A' for x in range(10000)]
group1 = np.concatenate((gr1_ones, gr1_zeros))
np.random.shuffle(group1)

In [219]:
# моделируем группу B
gr2_ones = np.ones(gr2_opened, dtype='int')
gr2_zeros = np.zeros(total-gr2_opened, dtype='int')
group2_label = ['group B' for x in range(10000)]
group2 = np.concatenate((gr2_ones, gr2_zeros))
np.random.shuffle(group2)

In [220]:
# создаем датафрейм
df1 = pd.DataFrame({'value': group1, 'group': group1_label})
df2 = pd.DataFrame({'value': group2, 'group': group2_label})

df = pd.concat([df1, df2], axis=0).reset_index(drop=True)

In [221]:
# перекрестная таблица
pd.crosstab(index=df.group, columns=df.value)

value,0,1
group,,
group A,7930,2070
group B,8000,2000


In [222]:
# воспользуемся chi square, т.к. имеем дело с дискретными данными
stat, p, dof, expected = chi2_contingency(pd.crosstab(index=df.group, columns=df.value))

In [223]:
# проводим t-test
alpha = 0.05  # уровень значимости

# вывод результата
if p <= alpha:
    print('Отклоняем нулевую гипотезу о равенстве двух групп. И принимаем альтернативную гипотезу о наличии статистически значимых различий.')
    if df[df['group'] == 'group A'].value.mean() > df[df['group'] == 'group B'].value.mean():
        print('Отдаем предпочтение группе A.')
    else:
        print('Отдаем предпочтение группе B.')
else:
    print('Не удалось выявить статистически значимые различия и отклонить нулевую гипотезу.\nНет разницы какой группе отдать предпочтение, т.к. они "равны".')

Не удалось выявить статистически значимые различия и отклонить нулевую гипотезу.
Нет разницы какой группе отдать предпочтение, т.к. они "равны".


### Задание 4 
Тестовой группе отправляется пуш с призывом совершить оплату, контрольной группе не отправляли.

|     | Размер группы  | Совершили оплату |
| ----- | ------------- | ------------- |
| Тест | 100 | 65  |
| Контроль | 100  | 50 |

**Является ли различие статистически значимым при уровне значимости 1%? А при 5%?**

#### Решение

In [224]:
# импорт библиотек
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

In [225]:
# задаем параметры групп
gr1_opened = 65
gr2_opened = 50
total = 100

In [226]:
# моделируем группу А
gr1_ones = np.ones(gr1_opened, dtype='int')
gr1_zeros = np.zeros(total - gr1_opened, dtype='int')
group1_label = ['Тест' for x in range(total)]
group1 = np.concatenate((gr1_ones, gr1_zeros))
np.random.shuffle(group1)

In [227]:
# моделируем группу B
gr2_ones = np.ones(gr2_opened, dtype='int')
gr2_zeros = np.zeros(total-gr2_opened, dtype='int')
group2_label = ['Контроль' for x in range(total)]
group2 = np.concatenate((gr2_ones, gr2_zeros))
np.random.shuffle(group2)

In [228]:
# создаем датафрейм
df1 = pd.DataFrame({'value': group1, 'group': group1_label})
df2 = pd.DataFrame({'value': group2, 'group': group2_label})

df = pd.concat([df1, df2], axis=0).reset_index(drop=True)

In [229]:
# перекрестная таблица
pd.crosstab(index=df.group, columns=df.value)

value,0,1
group,,
Контроль,50,50
Тест,35,65


In [230]:
# воспользуемся chi square, т.к. имеем дело с дискретными данными
stat, p, dof, expected = chi2_contingency(pd.crosstab(index=df.group, columns=df.value))

In [231]:
# проводим t-test
alphas = [0.05, 0.01]  # уровени значимости

# вывод результата
for alpha in alphas:
    print(f'Уровень статистической значимости alpha = {alpha}')
    if p <= alpha:
        print('Удалось получить статистически значимые различия. Отклоняем нулевую гипотезу о равенстве двух групп и принимаем альтернативную.\n')
    else:
        print('Не удалось выявить статистически значимые различия и отклонить нулевую гипотезу.\n')

Уровень статистической значимости alpha = 0.05
Удалось получить статистически значимые различия. Отклоняем нулевую гипотезу о равенстве двух групп и принимаем альтернативную.

Уровень статистической значимости alpha = 0.01
Не удалось выявить статистически значимые различия и отклонить нулевую гипотезу.



### Задание 5
Хотим протестировать новую версию сайта. Ожидаем, что вырастет число регистраций.  

Обычно дневная конверсия из захода на сайт в регистрацию - 50%. Хотим поймать изменение на 5 %-пунктов при уровне значимости 1% и мощности 80%.  

Так же известно, что каждый день к нам на сайт приходит 100 уникальных пользователей.  

**Какой размер выборки нам нужен? Сколько дней требуется проводить АВ тест?**

#### Решение

Воспользуемся калькулятором: https://abtestguide.com/abtestsize/  
Нам понадобится выборка размером 8022 уникальных посетителя.  
Длительность АВ теста составит 23 недели (161 день).
<img src='task5.png'>

### Задание 6 
|     | Размер группы  | Оставили заявку |
| ----- | ------------- | ------------- |
| Тест | 1000 | 600  |
| Контроль | 1000  | 500 |

Проводим обзвон клиентов (тест – звоним, контроль – нет).  

Цель – чтобы клиент оставил заявку на продукт.  

Средняя стоимость одного звонка 50 рублей (дозвонились или нет не важно), с одной заявки мы зарабатываем в среднем 180 рублей.  

**Проинтерпретируйте результаты АВ теста**

In [362]:
# импорт библиотек
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

In [363]:
# задаем параметры групп
gr1_opened = 600
gr2_opened = 500
total = 1000

In [364]:
# моделируем группу А
gr1_ones = np.ones(gr1_opened, dtype='int')
gr1_zeros = np.zeros(total - gr1_opened, dtype='int')
group1_label = ['Тест' for x in range(total)]
group1 = np.concatenate((gr1_ones, gr1_zeros))
np.random.shuffle(group1)

In [365]:
# моделируем группу B
gr2_ones = np.ones(gr2_opened, dtype='int')
gr2_zeros = np.zeros(total-gr2_opened, dtype='int')
group2_label = ['Контроль' for x in range(total)]
group2 = np.concatenate((gr2_ones, gr2_zeros))
np.random.shuffle(group2)

In [366]:
# создаем датафрейм
df1 = pd.DataFrame({'value': group1, 'group': group1_label})
df2 = pd.DataFrame({'value': group2, 'group': group2_label})

df = pd.concat([df1, df2], axis=0).reset_index(drop=True)

In [367]:
# добавляем расходы и доходы
df['expenses'] = 50
df['revenue'] = df.value * 180
df.loc[df['group'] == 'Контроль', ['expenses']] = 0
df['balance'] = df.revenue - df.expenses

In [368]:
# подсчет итоговой выручки
rev_test = df[df['group'] == 'Тест'].balance.sum()
rev_control = df[df['group'] == 'Контроль'].balance.sum()

In [369]:
# перекрестная таблица
pd.crosstab(index=df.group, columns=df.value)

value,0,1
group,,
Контроль,500,500
Тест,400,600


In [370]:
# воспользуемся chi square, т.к. имеем дело с дискретными данными
stat, p, dof, expected = chi2_contingency(pd.crosstab(index=df.group, columns=df.value))

In [371]:
# проводим t-test
alpha = 0.05  # уровени значимости

# вывод результата
if p <= alpha:
    print('Удалось получить статистически значимые различия. Отклоняем нулевую гипотезу о равенстве двух групп и принимаем альтернативную.')
    if rev_test < rev_control:
        print('Однако, выручка упала. Не будем запускать изменения в прод.')
    else:
        print('Удалось поднять выручку. Запускаем в прод.')
else:
    print('Не удалось выявить статистически значимые различия и отклонить нулевую гипотезу.')

Удалось получить статистически значимые различия. Отклоняем нулевую гипотезу о равенстве двух групп и принимаем альтернативную.
Однако, выручка упала. Не будем запускать изменения в прод.
